In [3]:
import requests
import util.item_analysis as ui
from urllib.parse import quote
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
# 예
market = ['심리 상담 센터']
addr = '서울시 서대문구 신촌동'

# 고객이 입력한 상품을 파는 매장을 리스트로 가져오기
market_list = market

In [4]:
# 고객이 입력한 주소 ['시', '구'] 또는 ['군', '면', '리'] 가져오기
addr = '서울시 서대문구 신촌동'

addr_list = addr.split(' ')
addr_list1 = []
addr_list2 = []

for i in addr_list:
    if i[-1] == '시' or i[-1] == '구' or i[-1] == '동':
        addr_list1.append(i)
    elif i[-1] == '군' or i[-1] == '면' or i[-1] == '리':
        addr_list2.append(i)

addr1 = ' '.join(addr_list1)
addr2 = ' '.join(addr_list2)

if addr1 != '':
    addr_result = addr1
else:
    addr_result = addr2

addr_result

'서울시 서대문구 신촌동'

In [5]:
# 고객이 입력한 주소 좌표 구하기
try:
    with open('keys/카카오api.txt') as file:
        kakao_key = file.read()

    base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
    header = {'Authorization': f'KakaoAK {kakao_key}'}
    url = f'{base_url}?query={quote(addr)}'
    result = requests.get(url, headers=header).json()
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])
except:
     print('주소 형식이 올바르지 않습니다.')

In [6]:
lat, lng

(37.5651616940519, 126.93927687604)

In [7]:
# 네이버 플레이스 셀레니움으로 들어가기
n_place_url = f'https://m.place.naver.com/place/list?x={lng}&y={lat}&query={market[0]} {addr}'
driver = webdriver.Chrome()
driver.get(n_place_url)

In [8]:
# '목록보기' 클릭
filter = driver.find_element(By.XPATH, '//*[@id="_place_portal_root"]/div/a') 
filter.click()
time.sleep(3)

In [9]:
# '거리순' 클릭
short_way = driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[2]/a') 
short_way.click()
time.sleep(3)

In [10]:
# '상세 주소 화살표' 클릭
try:
    juso_way = driver.find_elements(By.CLASS_NAME, 'uFxr1')
    juso_way[0].click()
    time.sleep(1)
except:
    print('근처에 매장이 없습니다')

In [11]:
# 마켓 정보 불러오기
soup = BeautifulSoup(driver.page_source)
market_list = soup.find('ul', class_='eDFz9')
mart = market_list.find('li', recursive=False)
mart

<li class="VLTHu" data-laim-exp-id="undefined"><div class="qbGlu"><div class="ouxiq"><a class="P7gyV" href="https://m.place.naver.com/place/1485341686?entry=ple" role="button"><div class="ApCpt"><div class="place_bluelink C6RjW"><span class="YwYLL">연세대학교 심리상담센터</span><span class="YzBgS">심리상담</span></div></div></a><a class="P7gyV" href="https://m.place.naver.com/place/1485341686?entry=ple" role="button"></a><div class="d7iiF"><div class="KgfA6"><span class="lWwyx NVngW"><span class="place_blind">현재 위치에서</span>160m</span><span class="lWwyx"><a aria-expanded="true" aria-haspopup="true" class="uFxr1" href="#" role="button"><span class="Pb4bU">서울 서대문구 신촌동</span><span class="_44_8"><span class="place_blind">상세주소 닫기</span><svg aria-hidden="true" class="Y2sSu" viewbox="0 0 13 8" xmlns="http://www.w3.org/2000/svg"><path d="M6.5 7.7L.1 1.4.8.7l5.6 5.6L12 .7l.7.7-6.2 6.3z"></path></svg></span></a></span><div><div class="AbTyi"><div class="zZfO1"><span class="MN3sj">도로명</span>서울특별시 서대문구 연세로 50 연세대

In [12]:
market_list.select_one('div.AbTyi').get_text().split('복사')[1][2:]

'서울특별시 서대문구 신촌동 134'

In [ ]:
results = []

# 네이버 플레이스 셀레니움으로 들어가기
n_place_url = f'https://m.place.naver.com/place/list?x={lng}&y={lat}&query={market[0]} {addr}'
driver = webdriver.Chrome()
driver.get(n_place_url)

# '목록보기' 클릭
filter = driver.find_element(By.XPATH, '//*[@id="_place_portal_root"]/div/a') 
filter.click()
time.sleep(3)

# '거리순' 클릭
short_way = driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[2]/a') 
short_way.click()
time.sleep(3)



# 마켓 정보 불러오기
soup = BeautifulSoup(driver.page_source, 'html.parser')
center_list = soup.find('ul', class_='eDFz9')

for i in range(10):
    if center_list:
        center = center_list.select('li', recursive=False)[i]

        if center:
            # 현재 위치에서 마켓 거리
            c_distance_element = center.select_one('span.lWwyx.NVngW')
            c_distance = c_distance_element.get_text().split('서')[1] if c_distance_element else 'N/A'

            # 마켓 이름
            if center.select_one('span.YwYLL'):
                c_title = center.select_one('span.YwYLL').get_text() 
            else:
                c_title = 'N/A'   

            # '상세 주소 화살표' 클릭
            try:
                juso_way = driver.find_elements(By.CLASS_NAME, 'uFxr1')
                juso_way[i].click()
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                center_list = soup.find('ul', class_='eDFz9')
                # 마켓 도로명 주소
                if center_list.select_one('div.zZfO1'):
                    if center_list.select_one('div.zZfO1').get_text()[:2] == '도로':
                        c_addr = center_list.select_one('div.zZfO1').get_text().split('복사')[0][3:]
                    elif center_list.select_one('div.zZfO1').get_text()[:2] == '지번':
                        c_addr = center_list.select_one('div.zZfO1').get_text().split('복사')[1][2:]
                    else:
                        c_addr = center_list.select_one('div.zZfO1').get_text().split('복사')[1][2:]
                else:
                    c_addr = 'N/A' 
                
                # 전화번호 추출
                if center.select_one('span.JsCty > a'):    
                    c_phone = center.select_one('span.JsCty > a').get('href')[4:]
                else:
                    c_addr = 'N/A' 

                

                
                time.sleep(1)
            except:
                print('근처에 매장이 없습니다')
                
            # 딕셔너리 형태로 저장
            center_data = {
                '거리': c_distance,
                '매장명': c_title,
                '주소': c_addr,
                '전화번호': c_phone
            }
            # 딕셔너리 형태로 저장한 것을 리스트에 저장 (3가지 마켓을 넣어야 하므로)
            results.append(center_data)

        else:
            print('마켓 정보를 찾을 수 없습니다.')
    else:
        print('마켓 리스트를 찾을 수 없습니다.')

results_df = pd.DataFrame(results)
print(results_df)


      거리             매장명                                      주소  \
0   160m    연세대학교 심리상담센터                 서울특별시 서대문구 연세로 50 연세대학교   
1   200m  연세대학교 아동가족상담센터                       서울특별시 서대문구 연세로 50   
2   510m        이혜련상담연구소             서울특별시 서대문구 성산로 488 이혜련상담연구소   
3   740m  호시담심리상담센터 연희동점       서울특별시 서대문구 연희로8길 18 스페이스독 3층 301호   
4   770m             인투뎀               서울특별시 서대문구 이화여대길 88-16 4층   
5   810m   바디마인드예술심리상담센터               서울특별시 서대문구 연희로 82 브라운스톤연희   
6   820m  체리나무숲 미술심리상담센터               서울특별시 서대문구 연세로 27 재유빌딩 5층   
7   970m   피어나다 언어심리상담센터  서울특별시 서대문구 이화여대1안길 3 3층 피어나다 언어심리상담 센터   
8  1.0km        참좋은마음연구소                서울특별시 서대문구 신촌로 121 1305호   
9  1.0km    쉬는숨 쉬는쉼 상담센터             서울특별시 서대문구 신촌로 109 르메이에르타운5   

                                                전화번호  
0  s://m.search.naver.com/search.naver?where=m&qu...  
1                                       02-2123-6483  
2                                     0507-1407-7745  
3                              

In [ ]:
center.select_one('span.JsCty > a').get('href')[4:]

In [ ]:
center_list.select_one('div.gkDbw')

In [ ]:
df = pd.DataFrame(results)
df

,거리,매장명,주소,전화번호
0,160m,연세대학교 심리상담센터,서울특별시 서대문구 연세로 50 연세대학교,s://m.search.naver.com/search.naver?where=m&qu...
1,200m,연세대학교 아동가족상담센터,서울특별시 서대문구 연세로 50,02-2123-6483
2,510m,이혜련상담연구소,서울특별시 서대문구 성산로 488 이혜련상담연구소,0507-1407-7745
3,740m,호시담심리상담센터 연희동점,서울특별시 서대문구 연희로8길 18 스페이스독 3층 301호,02-745-1052
4,770m,인투뎀,서울특별시 서대문구 이화여대길 88-16 4층,010-3273-5372
5,810m,바디마인드예술심리상담센터,서울특별시 서대문구 연희로 82 브라운스톤연희,0507-1319-1595
6,820m,체리나무숲 미술심리상담센터,서울특별시 서대문구 연세로 27 재유빌딩 5층,0507-1342-6399
7,970m,피어나다 언어심리상담센터,서울특별시 서대문구 이화여대1안길 3 3층 피어나다 언어심리상담 센터,02-6949-6212
8,1.0km,참좋은마음연구소,서울특별시 서대문구 신촌로 121 1305호,0507-1377-6287
9,1.0km,쉬는숨 쉬는쉼 상담센터,서울특별시 서대문구 신촌로 109 르메이에르타운5,0507-1406-9580


In [ ]:
# 고객이 입력한 주소 좌표 구하기
from tqdm import tqdm
with open('keys/카카오api.txt') as file:
    kakao_key = file.read()

base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization': f'KakaoAK {kakao_key}'}
lat_list, lng_list = [], []

for i in tqdm(df.index):
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))


100%|██████████| 10/10 [00:00<00:00, 10.09it/s]


In [ ]:
df['위도'] = lat_list
df['경도'] = lng_list
df

,거리,매장명,주소,전화번호,위도,경도
0,160m,연세대학교 심리상담센터,서울특별시 서대문구 연세로 50 연세대학교,s://m.search.naver.com/search.naver?where=m&qu...,37.566051,126.943916
1,200m,연세대학교 아동가족상담센터,서울특별시 서대문구 연세로 50,02-2123-6483,37.566051,126.943916
2,510m,이혜련상담연구소,서울특별시 서대문구 성산로 488 이혜련상담연구소,0507-1407-7745,37.560838,126.941356
3,740m,호시담심리상담센터 연희동점,서울특별시 서대문구 연희로8길 18 스페이스독 3층 301호,02-745-1052,37.565167,126.930936
4,770m,인투뎀,서울특별시 서대문구 이화여대길 88-16 4층,010-3273-5372,37.559446,126.944264
5,810m,바디마인드예술심리상담센터,서울특별시 서대문구 연희로 82 브라운스톤연희,0507-1319-1595,37.565213,126.930208
6,820m,체리나무숲 미술심리상담센터,서울특별시 서대문구 연세로 27 재유빌딩 5층,0507-1342-6399,37.558055,126.936714
7,970m,피어나다 언어심리상담센터,서울특별시 서대문구 이화여대1안길 3 3층 피어나다 언어심리상담 센터,02-6949-6212,37.557465,126.944481
8,1.0km,참좋은마음연구소,서울특별시 서대문구 신촌로 121 1305호,0507-1377-6287,37.556206,126.939296
9,1.0km,쉬는숨 쉬는쉼 상담센터,서울특별시 서대문구 신촌로 109 르메이에르타운5,0507-1406-9580,37.555831,126.937799


In [ ]:
df.to_csv('data/상담센터.csv')


In [ ]:
df = pd.read_csv('data/상담센터.csv')
df.head()

,Unnamed: 0,거리,매장명,주소,전화번호,위도,경도
0,0,160m,연세대학교 심리상담센터,서울특별시 서대문구 연세로 50 연세대학교,s://m.search.naver.com/search.naver?where=m&qu...,37.566051,126.943916
1,1,200m,연세대학교 아동가족상담센터,서울특별시 서대문구 연세로 50,02-2123-6483,37.566051,126.943916
2,2,510m,이혜련상담연구소,서울특별시 서대문구 성산로 488 이혜련상담연구소,0507-1407-7745,37.560838,126.941356
3,3,740m,호시담심리상담센터 연희동점,서울특별시 서대문구 연희로8길 18 스페이스독 3층 301호,02-745-1052,37.565167,126.930936
4,4,770m,인투뎀,서울특별시 서대문구 이화여대길 88-16 4층,010-3273-5372,37.559446,126.944264


In [ ]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df

,거리,매장명,주소,전화번호,위도,경도
0,160m,연세대학교 심리상담센터,서울특별시 서대문구 연세로 50 연세대학교,s://m.search.naver.com/search.naver?where=m&qu...,37.566051,126.943916
1,200m,연세대학교 아동가족상담센터,서울특별시 서대문구 연세로 50,02-2123-6483,37.566051,126.943916
2,510m,이혜련상담연구소,서울특별시 서대문구 성산로 488 이혜련상담연구소,0507-1407-7745,37.560838,126.941356
3,740m,호시담심리상담센터 연희동점,서울특별시 서대문구 연희로8길 18 스페이스독 3층 301호,02-745-1052,37.565167,126.930936
4,770m,인투뎀,서울특별시 서대문구 이화여대길 88-16 4층,010-3273-5372,37.559446,126.944264
5,810m,바디마인드예술심리상담센터,서울특별시 서대문구 연희로 82 브라운스톤연희,0507-1319-1595,37.565213,126.930208
6,820m,체리나무숲 미술심리상담센터,서울특별시 서대문구 연세로 27 재유빌딩 5층,0507-1342-6399,37.558055,126.936714
7,970m,피어나다 언어심리상담센터,서울특별시 서대문구 이화여대1안길 3 3층 피어나다 언어심리상담 센터,02-6949-6212,37.557465,126.944481
8,1.0km,참좋은마음연구소,서울특별시 서대문구 신촌로 121 1305호,0507-1377-6287,37.556206,126.939296
9,1.0km,쉬는숨 쉬는쉼 상담센터,서울특별시 서대문구 신촌로 109 르메이에르타운5,0507-1406-9580,37.555831,126.937799


In [ ]:
df = df.astype({'위도':'float', '경도':'float'})
df

,거리,매장명,주소,전화번호,위도,경도
0,160m,연세대학교 심리상담센터,서울특별시 서대문구 연세로 50 연세대학교,s://m.search.naver.com/search.naver?where=m&qu...,37.566051,126.943916
1,200m,연세대학교 아동가족상담센터,서울특별시 서대문구 연세로 50,02-2123-6483,37.566051,126.943916
2,510m,이혜련상담연구소,서울특별시 서대문구 성산로 488 이혜련상담연구소,0507-1407-7745,37.560838,126.941356
3,740m,호시담심리상담센터 연희동점,서울특별시 서대문구 연희로8길 18 스페이스독 3층 301호,02-745-1052,37.565167,126.930936
4,770m,인투뎀,서울특별시 서대문구 이화여대길 88-16 4층,010-3273-5372,37.559446,126.944264
5,810m,바디마인드예술심리상담센터,서울특별시 서대문구 연희로 82 브라운스톤연희,0507-1319-1595,37.565213,126.930208
6,820m,체리나무숲 미술심리상담센터,서울특별시 서대문구 연세로 27 재유빌딩 5층,0507-1342-6399,37.558055,126.936714
7,970m,피어나다 언어심리상담센터,서울특별시 서대문구 이화여대1안길 3 3층 피어나다 언어심리상담 센터,02-6949-6212,37.557465,126.944481
8,1.0km,참좋은마음연구소,서울특별시 서대문구 신촌로 121 1305호,0507-1377-6287,37.556206,126.939296
9,1.0km,쉬는숨 쉬는쉼 상담센터,서울특별시 서대문구 신촌로 109 르메이에르타운5,0507-1406-9580,37.555831,126.937799


In [ ]:
df.head()

,거리,매장명,주소,전화번호,위도,경도
0,160m,연세대학교 심리상담센터,서울특별시 서대문구 연세로 50 연세대학교,s://m.search.naver.com/search.naver?where=m&qu...,37.566051,126.943916
1,200m,연세대학교 아동가족상담센터,서울특별시 서대문구 연세로 50,02-2123-6483,37.566051,126.943916
2,510m,이혜련상담연구소,서울특별시 서대문구 성산로 488 이혜련상담연구소,0507-1407-7745,37.560838,126.941356
3,740m,호시담심리상담센터 연희동점,서울특별시 서대문구 연희로8길 18 스페이스독 3층 301호,02-745-1052,37.565167,126.930936
4,770m,인투뎀,서울특별시 서대문구 이화여대길 88-16 4층,010-3273-5372,37.559446,126.944264


In [ ]:
import folium
map = folium.Map([df.위도.mean(), df.경도.mean()], zoom_start=13)
map

In [ ]:
df.columns

Index(['거리', '매장명', '주소', '전화번호', '위도', '경도'], dtype='object')

In [ ]:
map = folium.Map([df.위도.mean(), df.경도.mean()], zoom_start=13)
for i in df.index:
    folium.Marker([df.위도[i], df.경도[i]], tooltip= f'<strong>{df.매장명[i]}</strong><br>{df.주소[i]}<br>{df.전화번호[i]}').add_to(map)

map


In [ ]:
def map_by_dong(df, dong):
    gf = df[df.BJDONG_NM.str.find(dong) == 0.0] 
    map = folium.Map([gf.위도.mean(), gf.경도.mean()], zoom_start=15)
    for i in gf.index:
        folium.Marker([df.위도[i], df.경도[i]], tooltip= [df.매장명[i], '<br>', df.주소[i]]).add_to(map)

    map.save('dong.html')

    map

In [ ]:
df.columns

Index(['거리', '매장명', '주소', '전화번호', '위도', '경도'], dtype='object')

In [ ]:
import folium
import pandas as pd

# 가상의 DataFrame을 생성하고 예시 데이터로 채웁니다.
center_data = {
                '거리': c_distance,
                '매장명': c_title,
                '주소': c_addr,
                '전화번호': c_phone
            }

# 지도 생성
map = folium.Map([df.위도.mean(), df.경도.mean()], zoom_start=14)

# DataFrame의 각 행에 대해 마커를 추가합니다.
for i in df.index:
    folium.Marker([df.위도[i], df.경도[i]], tooltip=[df.매장명[i], '<br>', df.주소[i], df.전화번호[i]]).add_to(map)

# 지도 저장
map.save('map.html')


In [ ]:
map